<a href="https://colab.research.google.com/github/redman157/HocML/blob/master/Logistic_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pson2900","key":"2507a7e3dd9a47a3b2ffc26308eaa233"}'}

In [2]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Jun  6 03:39 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# List available datasets.
!kaggle datasets list

ref                                                          title                                                size  lastUpdated          downloadCount  
-----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
vjchoudhary7/customer-segmentation-tutorial-in-python        Mall Customer Segmentation Data                       2KB  2018-08-11 07:23:02          15813  
fivethirtyeight/fivethirtyeight-comic-characters-dataset     FiveThirtyEight Comic Characters Dataset            577KB  2019-04-26 15:01:41           4798  
russellyates88/suicide-rates-overview-1985-to-2016           Suicide Rates Overview 1985 to 2016                 396KB  2018-12-01 19:18:25          40518  
karangadiya/fifa19                                           FIFA 19 complete player dataset                       2MB  2018-12-21 03:52:59          38556  
iarunava/cell-images-for-detecting-malaria                

In [6]:
# Copy the stackoverflow data set locally.
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 23.3MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 26.4MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.82MB/s]


In [107]:
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

training = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender =  pd.read_csv('gender_submission.csv')

X = training
y = training.Survived

feature_train = X.drop(['Name','PassengerId'],axis = 1)
gioitinh= {'male':0,'female':1}
feature_train.Sex.replace(gioitinh,inplace = True)

embarked = {'S':1,'C':2,'Q':3}
feature_train.Embarked.replace(embarked,inplace = True)
feature_train.Embarked.fillna(0,inplace = True)

feature_train['Family'] = feature_train.SibSp  + feature_train.Parch
feature_train.Family.loc[feature_train.Family > 0] =1 
feature_train.Family.loc[feature_train.Family == 0] = 0 

feature_train.Age.fillna(0,inplace = True)

feature_train.Cabin.fillna("X",inplace = True)
feature_train.Cabin = [i[0] for i in feature_train.Cabin]
s = feature_train.Cabin.sort_values().unique()
label = pd.factorize(s)

feature_train.Cabin = feature_train.Cabin.replace(label[1],label[0])
X = feature_train.drop(['Ticket','SibSp','Parch','Survived'],axis= 1)

X_train,X_val,y_train,y_val = train_test_split(X,y)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [108]:
test.head()
X_test = test
feature_test = X_test.drop(['Name','PassengerId'],axis = 1)

gioitinh= {'male':0,'female':1}
feature_test.Sex.replace(gioitinh,inplace = True)

embarked = {'S':0,'C':1,'Q':2}
feature_test.Embarked.replace(embarked,inplace= True)


feature_test['Family'] = feature_test.SibSp  + feature_train.Parch
feature_test.Family.loc[feature_test.Family > 0] =1 
feature_test.Family.loc[feature_test.Family == 0] = 0 

feature_test.Age.fillna(0,inplace = True)

feature_test.Cabin.fillna('X',inplace = True)
feature_test.Cabin = [i[0] for i in feature_test.Cabin]
h = feature_test.Cabin.sort_values().unique()
label = pd.factorize(h)
feature_test.Cabin = feature_test.Cabin.replace(label[1],label[0])

feature_test.Fare.fillna(0,inplace = True)
feature_test = feature_test.drop(['SibSp','Parch','Ticket'],axis = 1 )
X_test = feature_test
y_test = gender.Survived

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [109]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate
print(X_train.shape)
print(y_train.shape)

model = LogisticRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)
loss = mean_squared_error(y_test,pred)
print('Mean Squared Error: ', loss)
score = r2_score(y_test,pred)
print(f'Variabel accuracy: {score}')
vals = cross_validate(model,X_val,y_val,)
print('val_score \n', vals)
confu = confusion_matrix(pred,y_test)
print('confu', confu)

(668, 7)
(668,)
Mean Squared Error:  0.04784688995215311
Variabel accuracy: 0.793233082706767
val_score 
 {'fit_time': array([0.00303197, 0.00284863, 0.00264597]), 'score_time': array([0.00119662, 0.00101137, 0.00096774]), 'test_score': array([0.81333333, 0.78378378, 0.7972973 ])}
confu [[265  19]
 [  1 133]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Sp

In [0]:
import torch.nn as nn
import torch
import torch.optim as optimizer
y_train = y_train * -1.0

In [121]:
train_x = torch.Tensor(X_train.values)
train_y = torch.Tensor(y_train.values)
print(train_x.shape, train_y.shape)

torch.Size([668, 7]) torch.Size([668])


In [122]:
x_train = train_x.reshape([1,668*7]).squeeze()
print(x_train.shape)

torch.Size([4676])


In [0]:
model = nn.Sequential(
    nn.Linear(4676, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256,668),
    nn.Sigmoid()    
)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001

In [124]:
for epoch in range(50):
    # Forward Propagation
    y_pred = model(x_train)
    # Compute and print loss
    loss = criterion(train_y, y_pred)
    print('epoch: ', epoch,' loss: ', loss.item())
    # Zero the gradients
    optimizer.zero_grad()
    
    # perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()

IndexError: ignored

In [0]:
def sigmoid(X,derv = False):
  if(derv == True):
    return X * (1+X)
  return 1/(1 + np.exp(-X))
pred = sigmoid(np.dot(X_test,w))
def cost_function(X,y):
  return (-y* np.log(pred - ( y )* np.log(1-pred(X,w))))
def update_w(lr,epochs):
  batch_size = X.shape[1]
  w = np.matrix([0.0 for i in range(batch_size)])
  for i in range(epochs):
    error = np.repeat(pred - y.T,batch_size,axis = 0)
    error_derv = np.sum(np.matmul(error.T * X),axis = 1 )
    w = w - (lr/X_train.shape[0]) * error_derv
  return w

# update_w(0.01,100)
# def GD(X,y,lr):
#   epochs = X.shape[0]
#   batch_size = X.shape[1]
#   cost = [0.0 for i in range()]
#   for i in range(epochs):
#     cost = 

# fit_w(X_train,y_train,0.01,100) 
# error_dev = mse(pred - y_train)
  

In [0]:
batch_size = 5
nb_classes = 2
in_features = 10

model = nn.Linear(in_features, nb_classes)
criterion = nn.CrossEntropyLoss()

x = torch.randn(batch_size, in_features)
target = torch.empty(batch_size, dtype=torch.long).random_(nb_classes)

output = model(x)
loss = criterion(output, target)
loss.backward()